**Hello Community,**

**This notebook is destined to show viewers differents techniques to approach Plant Seeedling classifcation problem. We will implement image processing techniques and see its effect on the predictive modeling part, below are the techniques used in this work. This notebook is exported from Google colab, a few adjustement will be needed to run it on kaggle (images paths)**

**Image processing :**

**Data augmentation**

**Image morphological operations (erosion and dilation)**

**Image segmentation**

**Modeling :**

**Convolutional neural network based model**

**InceptionV3 based model**

**Support vector machine based model**

We will need this module later on for the preprocessing part

In [ ]:
!pip install -U tensorflow-addons


In [ ]:
from sklearn.model_selection import train_test_split
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
import keras
import random
import numpy as np
%matplotlib inline
import tqdm
import cv2
import os 
import tensorflow_addons as tfa
from keras import layers
import gc
from tensorflow.keras.regularizers import l2


Here we get a grasp of the data distribution, we notice a clear imbalance in the dataset. Some plant species are more frequent then other, thus this could lead to an imminent bias in the predictive. There will be a good probability to label an image as "Loose Silky" or "Common Chickweed" regardless of how the plant features. We don't have enough examples of how other species look in order to be able to infere confidently. In fact we need a larger dataset to enable the model to understand the right spatial characteristics and patterns of each plant specie. Especially in the cases where the problem at hand is multilabeled, the larger the number of labels is, the bigger the dataset should be so as to distinct between different species with ease. We could solve this problem by augmenting and balancing the dataset.

In [ ]:
training_images_files_names_black_grass = len(os.listdir("../input/plant-seedlings-classification/train/Black-grass"))
training_images_files_names_Charlock = len(os.listdir("../input/plant-seedlings-classification/train/Charlock"))
training_images_files_names_Cleavers = len(os.listdir("../input/plant-seedlings-classification/train/Cleavers"))
training_images_files_names_Common_Chickweed = len(os.listdir("../input/plant-seedlings-classification/train/Common Chickweed"))
training_images_files_names_Common_wheat = len(os.listdir("../input/plant-seedlings-classification/train/Common wheat"))
training_images_files_names_Scentless_Mayweed = len(os.listdir("../input/plant-seedlings-classification/train/Scentless Mayweed"))
training_images_files_names_Fat_Hen = len(os.listdir("../input/plant-seedlings-classification/train/Fat Hen"))
training_images_files_names_Loose_Silky_bent = len(os.listdir("../input/plant-seedlings-classification/train/Loose Silky-bent"))
training_images_files_names_Maize = len(os.listdir("../input/plant-seedlings-classification/train/Maize"))
training_images_files_names_Shepherds_Purse = len(os.listdir("../input/plant-seedlings-classification/train/Shepherds Purse"))
training_images_files_names_Small_flowered_Cranesbill = len(os.listdir("../input/plant-seedlings-classification/train/Small-flowered Cranesbill"))
training_images_files_names_Sugar_beet = len(os.listdir("../input/plant-seedlings-classification/train/Sugar beet"))
print("black grass sample size : ",training_images_files_names_black_grass,"\n")
print("Charlock sample size : ",training_images_files_names_Charlock,"\n")
print("Cleavers sample size : ",training_images_files_names_Cleavers,"\n")
print("Common Chickweed sample size : ",training_images_files_names_Common_Chickweed,"\n")
print("Common wheat sample size : ",training_images_files_names_Common_wheat,"\n")
print("Scentless Mayweed sample size : ",training_images_files_names_Scentless_Mayweed,"\n")
print("Fat Hen sample size : ",training_images_files_names_Fat_Hen,"\n")
print("Loose Silky sample size : ",training_images_files_names_Loose_Silky_bent,"\n")
print("Maize sample size : ",training_images_files_names_Maize,"\n")
print("Shepherds Purse sample size : ",training_images_files_names_Shepherds_Purse,"\n")
print("Small flowered Cranesbill sample size : ",training_images_files_names_Small_flowered_Cranesbill,"\n")
print("Sugar_beet sample size : ",training_images_files_names_Sugar_beet,"\n")
occurences = [training_images_files_names_black_grass/4750,training_images_files_names_Charlock/4750,training_images_files_names_Cleavers/4750,training_images_files_names_Common_Chickweed/4750,training_images_files_names_Common_wheat/4750,training_images_files_names_Fat_Hen/4750,training_images_files_names_Loose_Silky_bent/4750,training_images_files_names_Maize/4750,training_images_files_names_Scentless_Mayweed/4750,training_images_files_names_Shepherds_Purse/4750,training_images_files_names_Small_flowered_Cranesbill/4750,training_images_files_names_Sugar_beet/4750]
fig = plt.figure(figsize=[20,10])
ax = fig.add_axes([0,0,1,1])
ax.bar(["Black-grass","Charlock","Cleavers","Common Chickweed","Common wheat","Fat Hen","Loose Silky-bent","Maize","Scentless Mayweed","Shepherds Purse","Small-flowered Cranesbill","Sugar beet"],occurences)
plt.show()



the script below allows you to visualize a sample of 3 images of each plant species

In [ ]:
plant_species = os.listdir("../input/plant-seedlings-classification/train")
for plant_specie in plant_species:
   images_file_name = os.listdir("../input/plant-seedlings-classification/train/"+plant_specie)
   sample_size_to_display = random.sample(range(0,100), 3)
   if plant_specie == ".ipynb_checkpoints":
     continue
   print("plant species :",plant_specie)
   for i in range(0,3): 
     img = cv2.imread("../input/plant-seedlings-classification/train/"+plant_specie+"/"+images_file_name[sample_size_to_display[i]], cv2.IMREAD_COLOR)
     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
     plt.imshow(img)
     plt.show()
     #cv2_imshow(img)

We will be calling this function later on to visualize brut images and their corresponding processed versions

In [ ]:
def vizualize_random_images(original_tensor,transformed_tensor,sample_size_to_display):
     sample_size_to_display = random.sample(range(0,original_tensor.shape[0]), sample_size_to_display)
     for i in range(0,len(sample_size_to_display)): 
         img = original_tensor[sample_size_to_display[i]]
         mask = transformed_tensor[sample_size_to_display[i]]
         figure_size = 10
         print('Original image :')
         plt.figure(figsize=(figure_size,figure_size))
         plt.subplot(1,2,1)
         plt.imshow(img)
         plt.show()
         print('Transformed image :') 
         plt.figure(figsize=(figure_size,figure_size))
         plt.subplot(1,2,1)
         plt.imshow(mask)
         plt.show()

We proceed to read image data from their respective folders and store them in 4D array of size (number of image,100,100,3). We choose to resize image to the same dimension (100,100). Encode the species name following a one-hot encoding format, a matrix of size (number of images,12)

In [ ]:
# Labels encoding :
# black-grass : 0
# Charlock : 1
# Cleavers : 2
# Common_Chickweed : 3
# Common_wheat : 4
# Fat_Hen : 5
# Loose_Silky-bent : 6
# Maize : 7
# Scentless Mayweed : 8
# Shepherds Purse : 9
# Small flowered Cranesbill : 10
# Sugar beet : 11
label_encoding_mapper = {"Black-grass" : 0,"Charlock" : 1,"Cleavers": 2,"Common Chickweed" : 3,"Common wheat" : 4,"Fat Hen" : 5,"Loose Silky-bent" : 6,"Maize" : 7,"Scentless Mayweed" : 8,"Shepherds Purse" : 9,"Small-flowered Cranesbill" : 10,"Sugar beet" : 11}
training_size = training_images_files_names_black_grass + training_images_files_names_Charlock + training_images_files_names_Cleavers + training_images_files_names_Common_Chickweed + training_images_files_names_Common_wheat + training_images_files_names_Fat_Hen + training_images_files_names_Loose_Silky_bent + training_images_files_names_Maize + training_images_files_names_Scentless_Mayweed + training_images_files_names_Shepherds_Purse + training_images_files_names_Small_flowered_Cranesbill + training_images_files_names_Sugar_beet
y_train =  np.array([[0 for i in range(0,12)] for k in range(0,training_size)])
plant_species = os.listdir("../input/plant-seedlings-classification/train")
x_train = []
index = 0
for plant_specie in plant_species:
   images_file_name = os.listdir("../input/plant-seedlings-classification/train/"+plant_specie)
   for i in range(0,len(images_file_name)):
     img = cv2.imread("../input/plant-seedlings-classification/train/"+plant_specie+"/"+images_file_name[i], cv2.IMREAD_COLOR)
     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
     if img.shape != (100,100,3):
         resized = cv2.resize(img,(100,100), interpolation = cv2.INTER_AREA)
     x_train = x_train + [resized]
     y_train[index][label_encoding_mapper[plant_specie]] = 1 
     index = index + 1
x_train = np.array(x_train)
y_train = np.array(y_train)


In [ ]:
url = "../input/plant-seedlings-classification/sample_submission.csv"
submission_file = pd.read_csv(url)
test_images_file_names = list(submission_file["file"])

In [ ]:
x_test = []

for i in range(0,len(test_images_file_names)):
    img = cv2.imread("../input/plant-seedlings-classification/test/"+test_images_file_names[i], cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if img.shape != (100,100,3):
        resized = cv2.resize(img,(100,100), interpolation = cv2.INTER_AREA)
    x_test = x_test + [resized]
x_test = np.array(x_test)

In [ ]:
print("training sample : ",x_train.shape,"  ",y_train.shape,"\n")
print("test sample : ",x_test.shape,"\n")


Here we extract the images indices grouped by species from the array.

In [ ]:
plant_species= ["Black-grass","Charlock","Cleavers","Common Chickweed","Common wheat","Fat Hen","Loose Silky-bent","Maize","Scentless Mayweed","Shepherds Purse","Small-flowered Cranesbill","Sugar beet"]
index_storage = {"Black-grass" : [],"Charlock" : [],"Cleavers": [],"Common Chickweed" : [],"Common wheat" : [],"Fat Hen" : [],"Loose Silky-bent" : [],"Maize" : [],"Scentless Mayweed" : [],"Shepherds Purse" : [],"Small-flowered Cranesbill" : [],"Sugar beet" : []}
for i in range(0,len(x_train)):
    index = y_train[i].tolist().index(1)
    index_storage[plant_species[index]] = index_storage[plant_species[index]] + [i]

Then we apply random operations on images, so as to get a wider augmented dataset

In [ ]:
data_augmentation1 = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2)])
augmented_dataset1 = data_augmentation1(x_train)
gc.collect()
data_augmentation2 = tf.keras.Sequential([layers.RandomTranslation(height_factor=(-0.1,0.1),width_factor=(-0.1,0.1),interpolation="nearest")])
augmented_dataset2 = data_augmentation2(x_train)
gc.collect()
data_augmentation3 = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.3)])
augmented_dataset3 = data_augmentation3(x_train)
gc.collect()

Here we apply a processing procedure to each plant specie. We augment the specie sample up to a certain size specified by the parameter "Specie_total_size", the parameter "training_size" refers to the specie sample size. The augmented batches are concatenated to the 4D Tensor that embedds the original dataset.

In [ ]:

def augment_data(x_train,y_train,Specie,training_size,Specie_total_size):

  j = training_size
  one_hot_encoded_array_for_species = [0 for i in range(0,12)]
  for i in range(0,12):
      if label_encoding_mapper[Specie] == i:
           one_hot_encoded_array_for_species[i] = 1

  while j <= Specie_total_size:
     if j + training_size > Specie_total_size:
        b = training_size - (j + training_size - Specie_total_size)
        dummy_variable = random.randint(1,3)
        if dummy_variable == 1:
           augmented_dataset = data_augmentation1(x_train[index_storage[Specie]])
           gc.collect()
        if dummy_variable == 2:
           augmented_dataset = data_augmentation2(x_train[index_storage[Specie]])
           gc.collect()
        if dummy_variable == 3:
           augmented_dataset = data_augmentation3(x_train[index_storage[Specie]])
           gc.collect()
        x_train = np.concatenate([x_train,augmented_dataset[:b]])
        for i in range(0,b):
          y_train = y_train + [one_hot_encoded_array_for_species] 
     else:
        dummy_variable = random.randint(1,3)
        if dummy_variable == 1:
           augmented_dataset = data_augmentation1(x_train[index_storage[Specie]])
           gc.collect()
        if dummy_variable == 2:
           augmented_dataset = data_augmentation2(x_train[index_storage[Specie]])
           gc.collect()
        if dummy_variable == 3:
           augmented_dataset = data_augmentation3(x_train[index_storage[Specie]])
           gc.collect()
        x_train = np.concatenate([x_train,augmented_dataset])
        for i in range(0,training_size):
          y_train = y_train + [one_hot_encoded_array_for_species]
     j = j + training_size
     gc.collect()
     
  print(x_train.shape)
  


If we apply this procedure for every plant specie with the same parameter "Specie_total_size", we get a fully balanced dataset with augmented images. However "Specie_total_size" parameter shouldn't be too large due to the limited RAM resources available. the higher "Specie_total_size" is, the more computations should be made on the specie image batch, thus the more augmented images should be loaded into RAM. Each augmented batch consumes considerable memory space, therefore an overflow could be easily attained.

In [ ]:
augment_data(x_train,y_train,"Black-grass",len(index_storage["Black-grass"]),1000)

after augmenting black grass specie sample up to 1000 image we get a training set of size equal 5478 = 4750 (initial sample) + 728 (additional black grass augmented images)

# 1) Image processing
   # 1.1) Image blurring 

We start by applying gaussian filter to smoothen the image. As we can see the plant images contains many useless edges in the background, that are formed by the rocks. These edges would just create background noise, therefore we transform the discontinous pixels values into continous ones.  

In [ ]:
x_train_blurred = tfa.image.gaussian_filter2d(x_train,filter_shape=(4,4),sigma=100)
x_test_blurred = tfa.image.gaussian_filter2d(x_test,filter_shape=(4,4),sigma=100)
vizualize_random_images(x_train,x_train_blurred,2)


Our next step will consist of detecting the usefull information in the image, and eliminating the background. We need to construct a binary image (black and white) that indicate the object, that we wish to classify. We will be able to observe the shape of the usefull object alone, and the classification process will be much easier.
In order to achieve this, we convert RGB image to HSV format, a much helpful format to distinguish between colors. Luckily, the object detection problem at hand is simple one, because the object is clearly distinguishable from the background, no overlap so far (meaning no green background that could overlap with plants green color). Unfortunately, we would still have some noise ,imperfections in the image, thus as follow up, we erode the image a little bit ( to eliminate the isolated portions of the image that doesn't correspond to parts of the plant, noise in other words ) and then right after apply dilation to put more emphasis on the object shape.

In [ ]:
def segment_image(img): 
  img2 = cv2.cvtColor(np.float32(img),cv2.COLOR_RGB2HSV)
  green_segment_values = 50
  hsv_lower_bound = np.array([100-green_segment_values,0,0])
  hsv_upper_bound = np.array([100+green_segment_values,255,240])
  mask = cv2.inRange(img2,hsv_lower_bound,hsv_upper_bound)
  kernel = np.ones((3,3),np.uint8)
  mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
  return mask

  

We define a function that applies the processing pipeling to each image in the dataset and returns the processed dataset.

In [ ]:
def preprocess_dataset(x_train):
  preprocessed_data = []
  for i in range(0,x_train.shape[0]):
    preprocessed_data = preprocessed_data + [segment_image(x_train[i])]
  return preprocessed_data 

In [ ]:
preprocessed_data = np.array(preprocess_dataset(x_train_blurred))
preprocessed_test_data = np.array(preprocess_dataset(x_test_blurred))
vizualize_random_images(x_train,preprocessed_data,2)

We split the original dataset into training and validation

In [ ]:
(x_training, valX, y_training, valY) = train_test_split(x_train,y_train,test_size=0.15, random_state=10)


Then we embedd the training and validation set into an ImageDataGenerator object to generate batches of augmented images.

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,horizontal_flip=True,vertical_flip=True,rescale=1/255)
datagen.fit(x_training)
it = datagen.flow(x_training,y_training,batch_size=32)
datagenval = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
datagenval.fit(valX)
itval = datagenval.flow(valX, valY)
datagentest = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
datagentest.fit(x_test)
ittest = datagentest.flow(x_test)





We split the newly processed dataset and embedds into an ImageDataGenerator.

In [ ]:
(x_training_processed, valX_processed, y_training_processed, valY_processed) = train_test_split(preprocessed_data,y_train,test_size=0.15, random_state=10)



In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,horizontal_flip=True,vertical_flip=True,rescale=1/255)
x_training_processed = tf.reshape(x_training_processed,shape=(4037, 100, 100,1))
datagen.fit(x_training_processed)
training_processed_data_flow = datagen.flow(x_training_processed,y_training_processed)
datagenval = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valX_processed = tf.reshape(valX_processed,shape=(713, 100, 100,1))
datagenval.fit(valX_processed)
validation_processed_data_flow = datagenval.flow(valX_processed,valY_processed)
datagentest = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
preprocessed_test_data = tf.reshape(preprocessed_test_data,shape=(794, 100, 100,1))
datagentest.fit(preprocessed_test_data)
test_processed_data_flow = datagentest.flow(preprocessed_test_data)

We do this in order to have both instances of data available and be able to compare. Next we set the parameters, that will be used during training.

In [ ]:
INIT_LR = 1e-2 # initial learning rate
BATCH_SIZE = 32
EPOCHS = 150

s = tf.keras.backend.clear_session()  # clear default graph
# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)

opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR)
# prepare model for fitting (loss, optimizer, etc)
tqdm_callback = tfa.callbacks.TQDMProgressBar()


# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
     #return (INIT_LR*0.9)**epoch
   return (INIT_LR * 1.1)  
    #return INIT_LR
# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", tf.keras.backend.get_value(model.optimizer.lr))
class GarbageCollectorCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()


# 2) Modeling

Next we train multiple models on both versions of dataset.

In [ ]:
def make_svm_model():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())    
    model.add(tf.keras.layers.Dense(12,kernel_regularizer=l2(100),activation= 'linear'))
  
    return model

In [ ]:
s = tf.keras.backend.clear_session()
model = make_svm_model()
model.build(input_shape=(None,100,100,1))
model.summary()

model.compile(
    loss='categorical_hinge',
    optimizer=opt,  # for SGD
    metrics=[tfa.metrics.F1Score(num_classes=12,average="micro")]  # report accuracy during training
)
history = model.fit(
    training_processed_data_flow,  # prepared data
    epochs=150,
    validation_data=validation_processed_data_flow,
    #steps_per_epoch=len(x_training) // BATCH_SIZE,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(), 
               tqdm_callback,
               GarbageCollectorCallback(),
               ],
    shuffle=True
    ,verbose=0)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show() 

In [ ]:
def make_customized_cnn_model(image_number_channel):
    

    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides = (1, 1), padding="same", name = 'conv1', input_shape=(100,100,image_number_channel)))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    

    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides = (1, 1), padding="same", name = 'conv2'))   
    model.add(tf.keras.layers.MaxPooling2D((2, 2),padding="same",name='max_pool_1'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    
    
   
    

    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))


    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides = (1, 1), padding="same", name = 'conv3'))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
   
    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides = (1, 1), padding="same", name = 'conv4'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2),padding="same",name='max_pool_2'))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    
   
    
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    
    
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides = (1, 1), padding="same", name = 'conv5', input_shape=(100,100,1)))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
   
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides = (1, 1), padding="same", name = 'conv6', input_shape=(100,100,1))) 
    model.add(tf.keras.layers.MaxPooling2D((2, 2),padding="same",name='max_pool_3'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    
    

    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))

    model.add(tf.keras.layers.Flatten())    
    
    
    model.add(tf.keras.layers.Dense(256, name='fc1'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    model.add(tf.keras.layers.Dense(256, name='fc2'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))

    model.add(tf.keras.layers.Dense(12))    
    model.add(tf.keras.layers.Activation("softmax"))  
    return model

In [ ]:
s = tf.keras.backend.clear_session()
model = make_customized_cnn_model(3)
model.summary()
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model.compile(
    loss='categorical_crossentropy',  
    optimizer=opt,  
    metrics=[tfa.metrics.F1Score(num_classes=12,average="micro")] 
)
history = model.fit(
    it,  
    epochs=200,
    validation_data=itval,
    steps_per_epoch=len(x_training) // BATCH_SIZE,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(), 
               tqdm_callback,
               GarbageCollectorCallback(),
               ],
    shuffle=True
    ,verbose=0)

plt.plot(history.history['loss'],label="training loss")
plt.plot(history.history['val_loss'],label="validation loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show() 

In [ ]:
s = tf.keras.backend.clear_session()
model = make_customized_cnn_model(1)
model.summary()

model.compile(
    loss='categorical_crossentropy',  
    optimizer=opt,  
    metrics=[tfa.metrics.F1Score(num_classes=12,average="micro")]  
)
history = model.fit(
    training_processed_data_flow, 
    epochs=250,
    validation_data=validation_processed_data_flow,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(), 
               tqdm_callback,
               GarbageCollectorCallback(),
               ],
    shuffle=True
    ,verbose=0)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show() 

Inception model doesn't accept binary images as input. It always work with 3 channels

In [ ]:
def make_Inception_based_model():

    model = tf.keras.Sequential()
    
    model.add(tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(100,100,3)))
    model.add(tf.keras.layers.Flatten())    

    
    model.add(tf.keras.layers.Dense(256, name='fc1'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    model.add(tf.keras.layers.Dense(256, name='fc2'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    # FC
    model.add(tf.keras.layers.Dense(12))  # the last layer with neuron for each class    
    model.add(tf.keras.layers.Activation("softmax"))  # output probabilities
    return model
    
    
    
   
    
    
    

In [ ]:
s = tf.keras.backend.clear_session()
model = make_Inception_based_model()
model.summary()
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model.compile(
    loss='categorical_crossentropy',  
    optimizer=opt,  
    metrics=[tfa.metrics.F1Score(num_classes=12,average="micro")]  
)
history = model.fit(
    it,  # prepared data
    epochs=250,
    validation_data=itval,
    steps_per_epoch=len(x_training) // BATCH_SIZE,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(), 
               tqdm_callback,
               GarbageCollectorCallback(),
               ],
    shuffle=True
    ,verbose=0)

plt.plot(history.history['loss'],label="training loss")
plt.plot(history.history['val_loss'],label="validation loss")
plt.title('model loss')
plt.ylabel('loss')

In general the fluctuations observed in the loss function are explained by the use of Stochastic gradient descent optimizer, that makes use of sampling during optimization. the sample are generated from the training data at every iteration therefore the gradient formulas changes significantly from one iteration to another which generate sudden fluctuations.

save the model into google drive (or another location) via the specified path location, so you can load it back later and use it for future use. I saved three models : inception model, CNN model that trains on raw images, CNN models that trains on processed images, in google drive, then I used a simple python script that loads these models and use them for classification. The script file is named load_model.py : https://github.com/KarimAILab/Plant-seedling-classification.git

We can then infere on unseen images

In [ ]:
y_pred_test = model.predict(ittest)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

In brief, the models inceptions and cnn were the much more promosing.

That's it, thanks for giving time to this notebook, I hope it got you ready and inspired to create more advanced model!, feel free to ask anything in the comment section, or provide me with some more guidance or tips on how I could better present my work.

